In [1]:
import math
import sklearn
import numpy

import theano
import lasagne

import json

In [3]:
good = open("../scripts/codestyle_stats/fine_res.json", "r")
bad = open("../scripts/codestyle_stats/shit_res.json", "r")

In [4]:
bad_data = bad.read().replace('\n', ' ')
bad_dict = json.loads(bad_data)

In [5]:
good_data = good.read().replace('\n', ' ')
good_dict = json.loads(good_data)

In [7]:
print len(bad_dict)

608


In [8]:
print len(good_dict)

2540


In [ ]:
def build_network(input_var=None):
    l_in = lasagne.layers.InputLayer(shape=(None, 73, 5))